# Remaining analyses of the dataset
## 1. Library imports and setup

In [2]:
library(tidyverse)
library(lawstat)
library(PERMANOVA)
library(vegan)
library(npmv)

In [3]:
Sys.setenv(LANGUAGE="en")
Sys.setlocale("LC_TIME", "English")

theme_set(theme_bw())

[1] "English_United States.1252"

## 2. Import of the clean data

In [4]:
data.path <- "../../data"

file.name <- "df_chem_merged.rds"
df.chem.merged <- readRDS(paste0(data.path,"/",file.name))
head(df.chem.merged,3)

file.name <- "df_v_merged.rds"
df.v.merged <- readRDS(paste0(data.path,"/",file.name))
head(df.v.merged,3)

,ix,date_time,ec,temp,light,oxygen,pH,water_level,chlorophyll.a,chlorophyll.red,⋯,pH.upd,light.upd,salinity.upd,sensor.depth,depth,kelp,site,m.yr.factor,w.yr.factor,water.depth
,<chr>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<ord>,<fct>,<fct>,<ord>,<ord>,<dbl>
1,7m_G-1,2021-08-12 08:00:00,31194.1,4.71,688.9,13.50,8.35,216.510,6,11,⋯,NA,NA,NA,NA,7 [m],FALSE,GF,Aug21,W32-21,NA
2,7m_G-2,2021-08-12 08:30:00,31219.2,4.32,667.4,13.54,8.34,216.356,8,21,⋯,NA,NA,NA,NA,7 [m],FALSE,GF,Aug21,W32-21,NA
3,7m_G-3,2021-08-12 09:00:00,31319.6,5.29,796.5,13.31,8.35,179.936,6,54,⋯,NA,NA,NA,NA,7 [m],FALSE,GF,Aug21,W32-21,8.068685


,date_time,speed,heading,velocityN,velocityE,depth,site,kelp,m.yr.factor,w.yr.factor,depth.raw
,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<ord>,<fct>,<fct>,<ord>,<ord>,<fct>
1,2021-08-13 00:00:00,1.89,180.44,-1.89,-0.01,7 [m],GF,FALSE,Aug21,W33-21,7
2,2021-08-13 00:05:00,1.64,177.22,-1.63,0.08,7 [m],GF,FALSE,Aug21,W33-21,7
3,2021-08-13 00:10:00,1.85,177.76,-1.85,0.07,7 [m],GF,FALSE,Aug21,W33-21,7


#### Merging the velocity and physio-chemistry dataframes

In [5]:
df.velocity.30min <-  df.v.merged %>%
  mutate(date_time = floor_date(date_time, unit = "30 min")) %>%
  group_by(site,depth,date_time) %>%
  summarise(speed = mean(speed, na.rm = TRUE),
            velocityN = mean(velocityN, na.rm = TRUE),
            velocityE = mean(velocityE, na.rm = TRUE))

head(df.velocity.30min,3)

`summarise()` has grouped output by 'site', 'depth'. You can override using the
`.groups` argument.


site,depth,date_time,speed,velocityN,velocityE
<fct>,<ord>,<dttm>,<dbl>,<dbl>,<dbl>
GF,7 [m],2021-08-13 00:00:00,1.980000,-1.900000,-0.3050000
GF,7 [m],2021-08-13 00:30:00,2.693333,-1.281667,1.6366667
GF,7 [m],2021-08-13 01:00:00,1.900000,-1.053333,0.1516667


In [6]:
df.all2 <- merge(x=df.chem.merged, y= df.velocity.30min, by=c("site","depth","date_time"))
df.all2$ix <- NULL
head(df.all2)

,site,depth,date_time,ec,temp,light,oxygen,pH,water_level,chlorophyll.a,⋯,light.upd,salinity.upd,sensor.depth,kelp,m.yr.factor,w.yr.factor,water.depth,speed,velocityN,velocityE
,<fct>,<ord>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<ord>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>
1,GF,10 [m],2021-08-13 00:00:00,30120.4,5.25,64.6,13.17,8.37,NA,NA,⋯,64.6,32.9891,NA,FALSE,Aug21,W32-21,NA,2.918333,-1.3650000,2.348333
2,GF,10 [m],2021-08-13 00:30:00,30095.0,5.14,64.6,13.14,8.36,NA,NA,⋯,64.6,33.0673,NA,FALSE,Aug21,W33-21,NA,2.253333,0.4533333,2.181667
3,GF,10 [m],2021-08-13 01:00:00,30007.3,4.85,53.8,13.27,8.37,NA,NA,⋯,53.8,33.2496,NA,FALSE,Aug21,W33-21,NA,1.998333,0.9583333,1.751667
4,GF,10 [m],2021-08-13 01:30:00,30032.6,4.85,64.6,13.30,8.36,NA,NA,⋯,64.6,33.2807,NA,FALSE,Aug21,W33-21,NA,1.965000,1.2016667,1.550000
5,GF,10 [m],2021-08-13 02:00:00,29995.6,4.84,118.4,13.36,8.37,NA,NA,⋯,118.4,33.2453,NA,FALSE,Aug21,W33-21,NA,2.088333,1.4533333,1.441667
6,GF,10 [m],2021-08-13 02:30:00,30011.2,4.80,226.0,13.39,8.37,NA,NA,⋯,226.0,33.3047,NA,FALSE,Aug21,W33-21,NA,2.038333,1.8116667,0.740000


## 3. Performing t-test and Levene tests for sites with kelp and without kelp

Obviously the amount of the data is so large, that all differences will turn out statistically significant.

In [7]:
feat <- "ec"
x <- df.chem.merged %>% dplyr::filter(kelp==TRUE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
y <- df.chem.merged %>% dplyr::filter(kelp==FALSE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
t.test(x,y)

tmp.df <- df.chem.merged %>% dplyr::select(all_of(c(feat,"kelp"))) %>% na.omit()
x <- tmp.df %>% dplyr::select(all_of(feat)) %>% dplyr::pull()
g <- tmp.df %>% dplyr::select(all_of("kelp")) %>% dplyr::pull() %>% as.numeric() %>% factor()
levene.test(x,g)


	Welch Two Sample t-test

data:  x and y
t = -97.041, df = 133111, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -2180.229 -2093.902
sample estimates:
mean of x mean of y 
 22184.32  24321.39 



	Modified robust Brown-Forsythe Levene-type test based on the absolute
	deviations from the median

data:  x
Test Statistic = 9902.7, p-value < 2.2e-16


In [8]:
feat <- "temp"
x <- df.chem.merged %>% dplyr::filter(kelp==TRUE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
y <- df.chem.merged %>% dplyr::filter(kelp==FALSE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
t.test(x,y)

tmp.df <- df.chem.merged %>% dplyr::select(all_of(c(feat,"kelp"))) %>% na.omit()
x <- tmp.df %>% dplyr::select(all_of(feat)) %>% dplyr::pull()
g <- tmp.df %>% dplyr::select(all_of("kelp")) %>% dplyr::pull() %>% as.numeric() %>% factor()
levene.test(x,g)


	Welch Two Sample t-test

data:  x and y
t = 24.187, df = 142117, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.1944622 0.2287577
sample estimates:
mean of x mean of y 
 2.369895  2.158285 



	Modified robust Brown-Forsythe Levene-type test based on the absolute
	deviations from the median

data:  x
Test Statistic = 942.3, p-value < 2.2e-16


In [16]:
feat <- "light"
x <- df.chem.merged %>% dplyr::filter(kelp==TRUE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
y <- df.chem.merged %>% dplyr::filter(kelp==FALSE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
t.test(x,y)

tmp.df <- df.chem.merged %>% dplyr::select(all_of(c(feat,"kelp"))) %>% na.omit()
x <- tmp.df %>% dplyr::select(all_of(feat)) %>% dplyr::pull()
g <- tmp.df %>% dplyr::select(all_of("kelp")) %>% dplyr::pull() %>% as.numeric() %>% factor()
levene.test(x,g)


	Welch Two Sample t-test

data:  x and y
t = 35.494, df = 100850, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 26.32647 29.40393
sample estimates:
mean of x mean of y 
 79.57482  51.70962 



	Modified robust Brown-Forsythe Levene-type test based on the absolute
	deviations from the median

data:  x
Test Statistic = 2304.1, p-value < 2.2e-16


In [9]:
feat <- "pH"
x <- df.chem.merged %>% dplyr::filter(kelp==TRUE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
y <- df.chem.merged %>% dplyr::filter(kelp==FALSE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
t.test(x,y)

tmp.df <- df.chem.merged %>% dplyr::select(all_of(c(feat,"kelp"))) %>% na.omit()
x <- tmp.df %>% dplyr::select(all_of(feat)) %>% dplyr::pull()
g <- tmp.df %>% dplyr::select(all_of("kelp")) %>% dplyr::pull() %>% as.numeric() %>% factor()
levene.test(x,g)


	Welch Two Sample t-test

data:  x and y
t = -106.79, df = 113248, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.06319235 -0.06091451
sample estimates:
mean of x mean of y 
 8.212462  8.274516 



	Modified robust Brown-Forsythe Levene-type test based on the absolute
	deviations from the median

data:  x
Test Statistic = 109.26, p-value < 2.2e-16


In [18]:
feat <- "oxygen"
x <- df.chem.merged %>% dplyr::filter(kelp==TRUE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
y <- df.chem.merged %>% dplyr::filter(kelp==FALSE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
t.test(x,y)

tmp.df <- df.chem.merged %>% dplyr::select(all_of(c(feat,"kelp"))) %>% na.omit()
x <- tmp.df %>% dplyr::select(all_of(feat)) %>% dplyr::pull()
g <- tmp.df %>% dplyr::select(all_of("kelp")) %>% dplyr::pull() %>% as.numeric() %>% factor()
levene.test(x,g)


	Welch Two Sample t-test

data:  x and y
t = 16.313, df = 69579, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.06349744 0.08083921
sample estimates:
mean of x mean of y 
 14.10769  14.03552 



	Modified robust Brown-Forsythe Levene-type test based on the absolute
	deviations from the median

data:  x
Test Statistic = 150.24, p-value < 2.2e-16


In [10]:
feat <- "velocityN"
x <- df.v.merged %>% dplyr::filter(kelp==TRUE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
y <- df.v.merged %>% dplyr::filter(kelp==FALSE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
t.test(x,y)

tmp.df <- df.v.merged %>% dplyr::select(all_of(c(feat,"kelp"))) %>% na.omit()
x <- tmp.df %>% dplyr::select(all_of(feat)) %>% dplyr::pull()
g <- tmp.df %>% dplyr::select(all_of("kelp")) %>% dplyr::pull() %>% as.numeric() %>% factor()
levene.test(x,g)


	Welch Two Sample t-test

data:  x and y
t = -104.87, df = 1553538, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.5860659 -0.5645615
sample estimates:
 mean of x  mean of y 
-0.0588616  0.5164521 



	Modified robust Brown-Forsythe Levene-type test based on the absolute
	deviations from the median

data:  x
Test Statistic = 47202, p-value < 2.2e-16


In [11]:
feat <- "velocityE"
x <- df.v.merged %>% dplyr::filter(kelp==TRUE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
y <- df.v.merged %>% dplyr::filter(kelp==FALSE) %>% dplyr::select(all_of(feat)) %>% dplyr::pull() %>% na.omit()
t.test(x,y)

tmp.df <- df.v.merged %>% dplyr::select(all_of(c(feat,"kelp"))) %>% na.omit()
x <- tmp.df %>% dplyr::select(all_of(feat)) %>% dplyr::pull()
g <- tmp.df %>% dplyr::select(all_of("kelp")) %>% dplyr::pull() %>% as.numeric() %>% factor()
levene.test(x,g)


	Welch Two Sample t-test

data:  x and y
t = -128.56, df = 2943399, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.8705686 -0.8444217
sample estimates:
 mean of x  mean of y 
0.08955908 0.94705425 



	Modified robust Brown-Forsythe Levene-type test based on the absolute
	deviations from the median

data:  x
Test Statistic = 71415, p-value < 2.2e-16


## 4. Performing the Kruskal-Wallis rank sum test for groups with and without kelp forest presence

In [12]:
# oxygen
kruskal.test(oxygen ~ kelp, data = df.chem.merged)


	Kruskal-Wallis rank sum test

data:  oxygen by kelp
Kruskal-Wallis chi-squared = 87.002, df = 1, p-value < 2.2e-16


In [13]:
# pH
kruskal.test(pH ~ kelp, data = df.chem.merged)


	Kruskal-Wallis rank sum test

data:  pH by kelp
Kruskal-Wallis chi-squared = 10134, df = 1, p-value < 2.2e-16


In [14]:
# ec
kruskal.test(ec ~ kelp,  data = df.chem.merged)


	Kruskal-Wallis rank sum test

data:  ec by kelp
Kruskal-Wallis chi-squared = 9428.3, df = 1, p-value < 2.2e-16


In [15]:
# temperature
kruskal.test(temp ~ kelp,  data = df.chem.merged)


	Kruskal-Wallis rank sum test

data:  temp by kelp
Kruskal-Wallis chi-squared = 251.62, df = 1, p-value < 2.2e-16


In [16]:
# light
kruskal.test(light ~ kelp,  data = df.chem.merged)


	Kruskal-Wallis rank sum test

data:  light by kelp
Kruskal-Wallis chi-squared = 363.88, df = 1, p-value < 2.2e-16


In [17]:
# velocityN
kruskal.test(velocityN ~ kelp,  data = df.v.merged)


	Kruskal-Wallis rank sum test

data:  velocityN by kelp
Kruskal-Wallis chi-squared = 15396, df = 1, p-value < 2.2e-16


In [18]:
# velocityE
kruskal.test(velocityN ~ kelp,  data = df.v.merged)


	Kruskal-Wallis rank sum test

data:  velocityN by kelp
Kruskal-Wallis chi-squared = 15396, df = 1, p-value < 2.2e-16


## 4. PERMANOVA
### 4.1. Approach 1 - using `vegan` package

In [19]:
df.chem.merged$date_time <- round_date(df.chem.merged$date_time,unit='minute')  ## needs rounding by one second
df.velocity.30min <-  df.v.merged %>%
  mutate(date_time = floor_date(date_time, unit = "30 min")) %>%
  group_by(site,depth,date_time) %>%
  summarise(speed = mean(speed, na.rm = TRUE),
            velocityN = mean(velocityN, na.rm = TRUE),
            velocityE = mean(velocityE, na.rm = TRUE))

df.all2 <- merge(x=df.chem.merged, y= df.velocity.30min, by=c("site","depth","date_time"))
df.all2$ix <- NULL
head(df.all2)

`summarise()` has grouped output by 'site', 'depth'. You can override using the
`.groups` argument.


,site,depth,date_time,ec,temp,light,oxygen,pH,water_level,chlorophyll.a,⋯,light.upd,salinity.upd,sensor.depth,kelp,m.yr.factor,w.yr.factor,water.depth,speed,velocityN,velocityE
,<fct>,<ord>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<ord>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>
1,GF,10 [m],2021-08-13 00:00:00,30120.4,5.25,64.6,13.17,8.37,NA,NA,⋯,64.6,32.9891,NA,FALSE,Aug21,W32-21,NA,2.918333,-1.3650000,2.348333
2,GF,10 [m],2021-08-13 00:30:00,30095.0,5.14,64.6,13.14,8.36,NA,NA,⋯,64.6,33.0673,NA,FALSE,Aug21,W33-21,NA,2.253333,0.4533333,2.181667
3,GF,10 [m],2021-08-13 01:00:00,30007.3,4.85,53.8,13.27,8.37,NA,NA,⋯,53.8,33.2496,NA,FALSE,Aug21,W33-21,NA,1.998333,0.9583333,1.751667
4,GF,10 [m],2021-08-13 01:30:00,30032.6,4.85,64.6,13.30,8.36,NA,NA,⋯,64.6,33.2807,NA,FALSE,Aug21,W33-21,NA,1.965000,1.2016667,1.550000
5,GF,10 [m],2021-08-13 02:00:00,29995.6,4.84,118.4,13.36,8.37,NA,NA,⋯,118.4,33.2453,NA,FALSE,Aug21,W33-21,NA,2.088333,1.4533333,1.441667
6,GF,10 [m],2021-08-13 02:30:00,30011.2,4.80,226.0,13.39,8.37,NA,NA,⋯,226.0,33.3047,NA,FALSE,Aug21,W33-21,NA,2.038333,1.8116667,0.740000


In [20]:
featX <- c('oxygen','pH',"ec",'temp','light','velocityN','velocityE')
data.df <- na.omit(df.all2[featX])
has.kelp <- na.omit(df.all2[c(featX,"kelp")])$kelp
KELP <- data.frame(false=as.numeric(has.kelp==FALSE), true=as.numeric(has.kelp==TRUE))

adonis2(formula=KELP ~ oxygen+pH+ec+temp+light+velocityN+velocityE,
        data=data.df)

ERROR: Error: cannot allocate vector of size 226.4 Gb


### 4.2. Approach 2 - using `PERMANOVA` package

In [21]:
featX <- c('oxygen','pH',"ec",'temp','light','velocityN','velocityE')
data.df <- na.omit(df.all2[c(featX,'kelp')])
head(data.df,3)

,oxygen,pH,ec,temp,light,velocityN,velocityE,kelp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,13.17,8.37,30120.4,5.25,64.6,-1.3650000,2.348333,FALSE
2,13.14,8.36,30095.0,5.14,64.6,0.4533333,2.181667,FALSE
3,13.27,8.37,30007.3,4.85,53.8,0.9583333,1.751667,FALSE


In [22]:
featX <- c('oxygen','pH',"ec",'temp','light','velocityN','velocityE')
data.df <- na.omit(df.all2[featX])
has.kelp <- na.omit(df.all2[c(featX,"kelp")])$kelp

X = data.df 
X=IniTransform(X)
D = DistContinuous(X)
per.res=PERMANOVA(D, has.kelp) 
per.res

ERROR: Error: cannot allocate vector of size 452.8 Gb
